# Concat original transformed files

In [1]:
import pandas as pd
import os

In [2]:
os.chdir('/home/calvinhathcock/Documents/College/UNCC/Fall 2022/ITSC 4155/Project/transformed_data/csv')

In [3]:
files = os.listdir()

In [4]:
df_list = [pd.read_csv(x) for x in files]

# recursive function to concatenate dataframes 
def concat_df(df, idx):
    if idx == len(df_list) - 1:
        return df
    else:
        idx += 1
        df = pd.concat([df, concat_df(df_list[idx], idx)])
        return df

In [5]:
full_df = concat_df(df_list[0], 0)

In [6]:
full_df

,timestamp,count,building
0,2021-09-19 00:00:00,248,Atki
1,2021-09-19 01:00:00,80,Atki
2,2021-09-19 02:00:00,68,Atki
3,2021-09-19 03:00:00,26,Atki
4,2021-09-19 04:00:00,28,Atki
...,...,...,...
1782,2021-08-29 19:00:00,1134,Wood
1783,2021-08-29 20:00:00,486,Wood
1784,2021-08-29 21:00:00,548,Wood
1785,2021-08-29 22:00:00,414,Wood


In [7]:
full_df.building.value_counts()

Faci    8541
Robi    8541
Wood    8538
Came    8538
EPIC    8533
        ... 
PROS    4073
NCRC    3743
Nine    2845
Biss     902
Scie     278
Name: building, Length: 85, dtype: int64

In [8]:
os.chdir('/home/calvinhathcock/Documents/College/UNCC/Fall 2022/ITSC 4155/Project/ETL')

In [9]:
buildings = pd.read_csv('data_files/buildings.csv')

In [10]:
buildings.dropna(inplace=True)

In [11]:
buildings_list = list(buildings['Building abbreviation'])

In [12]:
full_df = full_df[ full_df.building.isin(buildings_list) ]

In [13]:
full_df.building.nunique()

65

### Append Longitude and Latitude 

In [14]:
buildings

,Building abbreviation,Building Name,Id,Latitude,Longitude
0,Atki,Atkins Library,1,35.305893,-80.732145
1,Auxi,Auxiliary Services Building,2,35.307772,-80.730500
3,Band,Barnhardt SAC,4,35.306302,-80.734409
4,BelG,Belk Gymnasium,5,35.305401,-80.735525
5,BelH,Belk Hall,6,35.310418,-80.734625
...,...,...,...,...,...
64,Unio,Union Deck,65,35.309127,-80.735300
66,Wall,Wallis Hall,67,35.311493,-80.733694
67,Winn,Winningham,68,35.305137,-80.730419
68,With,Witherspoon Hall,69,35.310855,-80.732324


In [17]:
lat_list = list(buildings.Latitude)

In [19]:
long_list = list(buildings.Longitude)

In [21]:
lat_dict = { k:v for (k,v) in zip(buildings_list, lat_list) }
long_dict = { k:v for (k,v) in zip(buildings_list, long_list) }

In [24]:
full_df['Latitude'] = full_df['building'].apply( lambda x: lat_dict[x] )

/tmp/ipykernel_11118/687435216.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df['Latitude'] = full_df['building'].apply( lambda x: lat_dict[x] )


In [26]:
full_df['Longitude'] = full_df['building'].apply( lambda x: long_dict[x] )

/tmp/ipykernel_11118/1631069350.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df['Longitude'] = full_df['building'].apply( lambda x: long_dict[x] )


In [27]:
full_df

,timestamp,count,building,Latitude,Longitude
0,2021-09-19 00:00:00,248,Atki,35.305893,-80.732145
1,2021-09-19 01:00:00,80,Atki,35.305893,-80.732145
2,2021-09-19 02:00:00,68,Atki,35.305893,-80.732145
3,2021-09-19 03:00:00,26,Atki,35.305893,-80.732145
4,2021-09-19 04:00:00,28,Atki,35.305893,-80.732145
...,...,...,...,...,...
1782,2021-08-29 19:00:00,1134,Wood,35.307516,-80.735717
1783,2021-08-29 20:00:00,486,Wood,35.307516,-80.735717
1784,2021-08-29 21:00:00,548,Wood,35.307516,-80.735717
1785,2021-08-29 22:00:00,414,Wood,35.307516,-80.735717


In [32]:
full_df = full_df.reindex(columns = ['timestamp', 'building', 'Latitude', 'Longitude', 'count'])

In [33]:
full_df.to_csv('data_files/full_data_flattened.csv', index = False)

In [34]:
full_df.shape

(532388, 5)